In [1]:
import pymysql
import tensorflow as tf
from sqlalchemy import create_engine
import pandas as pd
import json

# GPU 설정
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU 설정 완료:", gpus[0])
    except RuntimeError as e:
        print(e)

# 데이터베이스 연결 함수
def get_db():
    db = pymysql.connect(
        host='human-mysql.mysql.database.azure.com',  # Azure MySQL Host
        port=3306,  # Port number (MySQL default is 3306)
        user='human',  # Username
        passwd='!q1w2e3r4',  # Password
        db='humandb',  # Database name
        ssl_ca=r'/home/azureuser/Desktop/config/DigiCertGlobalRootG2.crt.pem'  # SSL certificate path
    )
    return db

# DB에서 데이터 로드
db_connection = get_db()
cursor = db_connection.cursor()

query = "SELECT * FROM humandb.modeling_retrival"
cursor.execute(query)
columns = [desc[0] for desc in cursor.description]  # 컬럼 이름 가져오기
all_data = cursor.fetchall()  # 데이터 가져오기

# Pandas DataFrame으로 변환
df = pd.DataFrame(all_data, columns=columns)
# 데이터 확인
print("데이터셋의 크기:", df.shape)
print("컬럼명:", df.columns)

2025-02-05 01:34:30.266655: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-05 01:34:31.589140: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2025-02-05 01:34:31.589233: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2025-02-05 01:34:31.589240: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] 

GPU 설정 완료: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
데이터셋의 크기: (100603, 2)
컬럼명: Index(['features_json', 'Reason'], dtype='object')


In [2]:
def parse_json(x):
    if isinstance(x, str):  # JSON이 문자열로 저장된 경우 변환
        return json.loads(x)
    return x  # 이미 리스트 형식이라면 그대로 반환

df["features_vector"] = df["features_json"].apply(parse_json)

# features_json 컬럼 삭제 (벡터 리스트만 유지)
df = df.drop(columns=["features_json"])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100603 entries, 0 to 100602
Data columns (total 2 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Reason           100603 non-null  object
 1   features_vector  100603 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB


In [4]:
df.head()

,Reason,features_vector
0,특이사항 없음으로 일상 상황으로 판단,"[73.0, 16.0, 98.0, 0.0, 9.0, 0.0, 0.0, 0.0, 10..."
1,특이사항 없음으로 일상 상황으로 판단,"[73.0, 16.0, 98.0, 0.0, 9.0, 0.0, 0.0, 0.0, 0...."
2,특이사항 없음으로 일상 상황으로 판단,"[73.0, 16.0, 98.0, 0.0, 9.0, 0.0, 0.0, 0.0, 0...."
3,특이사항 없음으로 일상 상황으로 판단,"[74.0, 16.0, 98.0, 0.0, 9.0, 0.0, 0.0, 0.0, 0...."
4,특이사항 없음으로 일상 상황으로 판단,"[74.0, 16.0, 98.0, 0.0, 9.0, 0.0, 0.0, 0.0, 0...."


In [5]:
import faiss
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

# 데이터셋을 학습용(80%)과 테스트용(20%)으로 분할
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# FAISS 인덱스 생성 (L2 거리 기반)
vector_data = np.array(train_df["features_vector"].tolist(), dtype=np.float32)
index = faiss.IndexFlatL2(vector_data.shape[1])
index.add(vector_data)

# 테스트 데이터 검색
test_vectors = np.array(test_df["features_vector"].tolist(), dtype=np.float32)
true_labels = test_df["Reason"].tolist()

# FAISS 검색 (Top-1)
_, nearest = index.search(test_vectors, 1)

# 검색된 Reason 값 가져오기
retrieved_reasons = [train_df.iloc[i]["Reason"] for i in nearest.flatten()]

# 정확도(Accuracy), 정밀도(Precision), 재현율(Recall) 계산
accuracy = accuracy_score(true_labels, retrieved_reasons)
precision = precision_score(true_labels, retrieved_reasons, average="macro")
recall = recall_score(true_labels, retrieved_reasons, average="macro")

# 결과 출력
print(f"정확도(Accuracy): {accuracy:.2f}")
print(f"정밀도(Precision): {precision:.2f}")
print(f"재현율(Recall): {recall:.2f}")

정확도(Accuracy): 0.82
정밀도(Precision): 0.19
재현율(Recall): 0.16


/home/azureuser/anaconda3/envs/emergency/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
# 올바른 import 방식
from imblearn.over_sampling import SMOTE

# 특정 클래스를 제거 (샘플 개수가 5개 이하인 클래스 삭제)
min_samples_required = 2
filtered_df = df[df.groupby("Reason")["Reason"].transform("count") >= min_samples_required]

# SMOTE 적용
smote = SMOTE(sampling_strategy="auto", random_state=42, k_neighbors=1)
X_resampled, y_resampled = smote.fit_resample(np.array(filtered_df["features_vector"].tolist()), filtered_df["Reason"])

# 최종 균형 잡힌 데이터 생성
final_df = pd.DataFrame({"features_vector": X_resampled.tolist(), "Reason": y_resampled})

print(final_df["Reason"].value_counts())  # 클래스 분포 확인


Reason
특이사항 없음으로 일상 상황으로 판단               78803
분당 호흡, 혈중산소농도 등 기준으로 주의 상황으로 판단    78803
피부온도변화, 응급버튼 기준으로 위험 상황으로 판단       78803
혈중산소농도 기준으로 위험 상황으로 판단             78803
응급음성 기준으로 위험 상황으로 판단               78803
응급버튼 기준으로 위험 상황으로 판단               78803
분당 심박, 피부온도변화 기준으로 위험 상황으로 판단      78803
분당 호흡, 피부온도변화 기준으로 위험 상황으로 판단      78803
분당 심박, 혈중산소농도 등 기준으로 주의 상황으로 판단    78803
분당 심박, 피부온도변화 등 기준으로 주의 상황으로 판단    78803
분당 심박 기준으로 위험 상황으로 판단              78803
분당 심박, 분당 호흡 등 기준으로 위험 상황으로 판단     78803
분당 심박, 분당 호흡 기준으로 위험 상황으로 판단       78803
분당 심박, 혈중산소농도 기준으로 주의 상황으로 판단      78803
분당 심박, 분당 호흡 등 기준으로 주의 상황으로 판단     78803
혈중산소농도, 피부온도변화 기준으로 주의 상황으로 판단     78803
분당 심박, 피부온도변화 기준으로 주의 상황으로 판단      78803
피부온도변화 기준으로 위험 상황으로 판단             78803
피부온도변화 기준으로 주의 상황으로 판단             78803
혈중산소농도 기준으로 주의 상황으로 판단             78803
분당 호흡, 혈중산소농도 기준으로 주의 상황으로 판단      78803
분당 호흡 기준으로 주의 상황으로 판단              78803
분당 호흡, 피부온도변화 기준으로 주의 상황으로 판단      78803
분당 호흡 기준으로 위험 상황으로 판단              78803
분당 심박 기준으

In [7]:
import pandas as pd

# 각 클래스별 최대 10,000개만 유지
max_samples_per_class = 15000
df_sampled = final_df.groupby("Reason").apply(lambda x: x.sample(n=min(len(x), max_samples_per_class), random_state=42)).reset_index(drop=True)

# 샘플링 후 클래스 분포 확인
print(df_sampled["Reason"].value_counts())

Reason
분당 심박 기준으로 위험 상황으로 판단              15000
분당 호흡, 피부온도변화 기준으로 위험 상황으로 판단      15000
혈중산소농도 기준으로 주의 상황으로 판단             15000
혈중산소농도 기준으로 위험 상황으로 판단             15000
피부온도변화, 응급버튼 기준으로 위험 상황으로 판단       15000
피부온도변화 기준으로 주의 상황으로 판단             15000
피부온도변화 기준으로 위험 상황으로 판단             15000
특이사항 없음으로 일상 상황으로 판단               15000
응급음성 기준으로 위험 상황으로 판단               15000
응급버튼 기준으로 위험 상황으로 판단               15000
분당 호흡, 혈중산소농도 등 기준으로 주의 상황으로 판단    15000
분당 호흡, 혈중산소농도 기준으로 주의 상황으로 판단      15000
분당 호흡, 피부온도변화 기준으로 주의 상황으로 판단      15000
분당 호흡 기준으로 주의 상황으로 판단              15000
분당 심박 기준으로 주의 상황으로 판단              15000
분당 호흡 기준으로 위험 상황으로 판단              15000
분당 심박, 혈중산소농도 등 기준으로 주의 상황으로 판단    15000
분당 심박, 혈중산소농도 기준으로 주의 상황으로 판단      15000
분당 심박, 혈중산소농도 기준으로 위험 상황으로 판단      15000
분당 심박, 피부온도변화 등 기준으로 주의 상황으로 판단    15000
분당 심박, 피부온도변화 기준으로 주의 상황으로 판단      15000
분당 심박, 피부온도변화 기준으로 위험 상황으로 판단      15000
분당 심박, 분당 호흡 등 기준으로 주의 상황으로 판단     15000
분당 심박, 분당 호흡 등 기준으로 위험 상황으로 판단     15000
분당 심박, 분당

In [8]:
import faiss
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

# 데이터셋을 학습용(80%)과 테스트용(20%)으로 분할
train_df, test_df = train_test_split(df_sampled, test_size=0.2, random_state=42)

# FAISS CPU 인덱스 생성
vector_data = np.array(train_df["features_vector"].tolist(), dtype=np.float32)

# CPU에서 FAISS 실행 (메모리 부족 방지)
cpu_index = faiss.IndexFlatL2(vector_data.shape[1])
cpu_index.add(vector_data)

# 테스트 데이터 검색
test_vectors = np.array(test_df["features_vector"].tolist(), dtype=np.float32)
true_labels = test_df["Reason"].tolist()

# FAISS 검색 (Top-1 검색)
_, nearest = cpu_index.search(test_vectors, 1)

# 검색된 Reason 값 가져오기
retrieved_reasons = [train_df.iloc[i]["Reason"] for i in nearest.flatten()]

# 정확도(Accuracy), 정밀도(Precision), 재현율(Recall) 계산
accuracy = accuracy_score(true_labels, retrieved_reasons)
precision = precision_score(true_labels, retrieved_reasons, average="macro")
recall = recall_score(true_labels, retrieved_reasons, average="macro")

# 결과 출력
print(f"정확도(Accuracy): {accuracy:.4f}")
print(f"정밀도(Precision): {precision:.4f}")
print(f"재현율(Recall): {recall:.4f}")


정확도(Accuracy): 0.9912
정밀도(Precision): 0.9912
재현율(Recall): 0.9910


In [9]:
# 훈련 데이터와 테스트 데이터가 중복되는지 확인
duplicate_count = test_df[test_df.index.isin(train_df.index)].shape[0]

print(f"훈련 데이터와 테스트 데이터의 중복 개수: {duplicate_count}")


훈련 데이터와 테스트 데이터의 중복 개수: 0


In [10]:
# 학습 데이터와 테스트 데이터의 클래스 분포 확인
print("Train 데이터 클래스 분포:")
print(train_df["Reason"].value_counts(normalize=True))

print("\n Test 데이터 클래스 분포:")
print(test_df["Reason"].value_counts(normalize=True))

Train 데이터 클래스 분포:
Reason
분당 호흡, 혈중산소농도 등 기준으로 주의 상황으로 판단    0.037386
분당 심박 기준으로 주의 상황으로 판단              0.037370
분당 심박, 피부온도변화 등 기준으로 주의 상황으로 판단    0.037238
피부온도변화, 응급버튼 기준으로 위험 상황으로 판단       0.037182
특이사항 없음으로 일상 상황으로 판단               0.037179
분당 호흡, 피부온도변화 기준으로 위험 상황으로 판단      0.037176
분당 호흡, 피부온도변화 기준으로 주의 상황으로 판단      0.037151
피부온도변화 기준으로 주의 상황으로 판단             0.037127
분당 심박, 분당 호흡 등 기준으로 위험 상황으로 판단     0.037108
응급버튼 기준으로 위험 상황으로 판단               0.037108
혈중산소농도, 피부온도변화 기준으로 주의 상황으로 판단     0.037086
분당 심박, 분당 호흡 기준으로 주의 상황으로 판단       0.037068
분당 호흡 기준으로 위험 상황으로 판단              0.037049
응급음성 기준으로 위험 상황으로 판단               0.037003
혈중산소농도 기준으로 위험 상황으로 판단             0.036997
분당 심박 기준으로 위험 상황으로 판단              0.036994
혈중산소농도 기준으로 주의 상황으로 판단             0.036991
분당 심박, 혈중산소농도 등 기준으로 주의 상황으로 판단    0.036988
분당 심박, 혈중산소농도 기준으로 위험 상황으로 판단      0.036951
분당 심박, 분당 호흡 기준으로 위험 상황으로 판단       0.036948
분당 호흡 기준으로 주의 상황으로 판단              0.036932
분당 호흡, 혈중산소농도 기준으로 주의 상황으로 판단      0.036914
분당 심박, 

In [ ]:
from sklearn.decomposition import PCA
import faiss
import numpy as np


# PCA 적용 (예: 10차원 → 5차원)
pca = PCA(n_components=5, random_state=42)
train_vectors_pca = pca.fit_transform(vector_data)
test_vectors_pca = pca.transform(test_vectors)

# FAISS가 처리할 수 있도록 NumPy 배열을 C-contiguous로 변환
train_vectors_pca = np.ascontiguousarray(train_vectors_pca, dtype=np.float32)
test_vectors_pca = np.ascontiguousarray(test_vectors_pca, dtype=np.float32)

# FAISS에 PCA 적용된 데이터 추가
cpu_index_pca = faiss.IndexFlatL2(train_vectors_pca.shape[1])
cpu_index_pca.add(train_vectors_pca)  # 이제 오류 없이 실행 가능


# 다시 검색
_, nearest_pca = cpu_index_pca.search(test_vectors_pca, 1)
retrieved_reasons_pca = [train_df.iloc[i]["Reason"] for i in nearest_pca.flatten()]


In [12]:
#  정확도(Accuracy), 정밀도(Precision), 재현율(Recall) 계산
accuracy = accuracy_score(true_labels, retrieved_reasons_pca)
precision = precision_score(true_labels, retrieved_reasons_pca, average="macro")
recall = recall_score(true_labels, retrieved_reasons_pca, average="macro")

#  결과 출력
print(f"정확도(Accuracy): {accuracy:.4f}")
print(f"정밀도(Precision): {precision:.4f}")
print(f"재현율(Recall): {recall:.4f}")

정확도(Accuracy): 0.9763
정밀도(Precision): 0.9762
재현율(Recall): 0.9761


In [21]:
import faiss
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score, precision_score, recall_score

# 데이터셋을 학습용(80%)과 테스트용(20%)으로 분할
train_df, test_df = train_test_split(df_sampled, test_size=0.2, random_state=42, shuffle=True)

# NumPy 배열로 변환
vector_data = np.array(train_df["features_vector"].tolist(), dtype=np.float32)
test_vectors = np.array(test_df["features_vector"].tolist(), dtype=np.float32)

# L2 정규화 (코사인 유사도 사용)
vector_data = normalize(vector_data, axis=1)  # 훈련 데이터 정규화
test_vectors = normalize(test_vectors, axis=1)  # 테스트 데이터 정규화

# FAISS 코사인 유사도 기반 인덱스 생성 (Inner Product 사용)
cpu_index = faiss.IndexFlatIP(vector_data.shape[1])  # Inner Product (IP) 인덱스 사용
cpu_index.add(vector_data)  # 훈련 데이터 추가

# 테스트 데이터 검색 (Top-1 검색)
_, nearest = cpu_index.search(test_vectors, 1)

# 검색된 Reason 값 가져오기
retrieved_reasons = [train_df.iloc[i]["Reason"] for i in nearest.flatten()]
true_labels = test_df["Reason"].tolist()

# 정확도(Accuracy), 정밀도(Precision), 재현율(Recall) 계산
accuracy = accuracy_score(true_labels, retrieved_reasons)
precision = precision_score(true_labels, retrieved_reasons, average="macro", zero_division=1)
recall = recall_score(true_labels, retrieved_reasons, average="macro", zero_division=1)

# 결과 출력
print(f"코사인 유사도 기반 정확도(Accuracy): {accuracy:.4f}")
print(f"코사인 유사도 기반 정밀도(Precision): {precision:.4f}")
print(f"코사인 유사도 기반 재현율(Recall): {recall:.4f}")


✅ 코사인 유사도 기반 정확도(Accuracy): 0.9907
✅ 코사인 유사도 기반 정밀도(Precision): 0.9908
✅ 코사인 유사도 기반 재현율(Recall): 0.9905
